In [1]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt

# Introduction
For the following calculations the drag force is ignored wicht leads to the eqaution of motion as

$$
m \cdot \ddot{x} = T - W
$$

where 

$$
m = m_0 - t \cdot \dot{m}
$$

and

$$
W = m \cdot g
$$

and

$$
T = c_e \cdot \dot{m} = const
$$

which leads to

$$
\dot{m} = \frac{T}{c_e}
$$.

For the analysis three dimensionless values are consitiondered

$$
\tilde{m}_f = \frac{m_f}{m_0}
$$

where $m_0$ is the initial plane mass and $m_f$ is the fuel mass.
Additionaly

$$
\tilde{T} = \frac{T}{m_0 \cdot g}
$$

and

$$
t_e \cdot \dot{m} = m_f \rightarrow t_e =  \frac{c_e \cdot m_f}{T}
$$

which leads to

$$
t_e = \frac{c_e}{g} \frac{\tilde{m}_f}{\tilde{T}}
$$

and

$$
\tilde{t} = \frac{t}{t_e} 
$$

and

$$
\tilde{\ddot{x}} = \frac{\ddot{x}}{g}.
$$

# Dimensionless equation

The above to the dimensionless equation

$$
\tilde{\ddot{x}} = \frac{\tilde{T}}{1 - \tilde{t} \cdot \tilde{m}_f} - 1.
$$

using 

$$
\frac{d v}{d t} = \tilde{\ddot{x}} \cdot g
$$

and

$$
\frac{d v}{d t} = \frac{d v}{d \tilde{t}} \frac{d \tilde{t}}{d t} = \frac{1}{t_e} \frac{d v}{d \tilde{t}}
$$

leads to

$$
\underbrace{\frac{1}{t_e \cdot g} v}_{\tilde{v}} = \int \tilde{\ddot{x}} \mathrm{d}\tilde{t}
$$

and

$$
\underbrace{\frac{1}{t_e^2 \cdot g} s}_{\tilde{s}} = \int \tilde{v} \mathrm{d}\tilde{t}
$$

In [2]:
t, T, m, q, H = symbols("ttilde Ttilde mtilde_f qtilde Htilde")

In [3]:
xddot = T / (1 - t*m) - 1

In [4]:
xddot

Ttilde/(-mtilde_f*ttilde + 1) - 1

In [5]:
xdot = -(T/m)*log(1 - m*t) - t

In [6]:
xdot

-Ttilde*log(-mtilde_f*ttilde + 1)/mtilde_f - ttilde

In [7]:
x = (m*t*(-m + T) + (T - m*t*T)*log(1 - m*t))/(m**2)

In [8]:
x.subs(t, 1)

(mtilde_f*(Ttilde - mtilde_f) + (-Ttilde*mtilde_f + Ttilde)*log(1 - mtilde_f))/mtilde_f**2

In [9]:
x0 = x.subs(t, 1)

In [10]:
xdot0 = xdot.subs(t, 1)

In [11]:
s = -Rational(1, 2)*t**2 + xdot0*t + x0

In [12]:
s

-ttilde**2/2 + ttilde*(-Ttilde*log(1 - mtilde_f)/mtilde_f - 1) + (mtilde_f*(Ttilde - mtilde_f) + (-Ttilde*mtilde_f + Ttilde)*log(1 - mtilde_f))/mtilde_f**2

In [13]:
tmax = solve(diff(s, t), t)[0]

In [14]:
tmax

-(Ttilde*log(1 - mtilde_f) + mtilde_f)/mtilde_f

In [15]:
smax = s.subs(t, tmax).simplify()

In [16]:
sh = smax/H

In [17]:
sh

(Ttilde**2*log(1 - mtilde_f)**2/2 + Ttilde*mtilde_f + Ttilde*log(1 - mtilde_f) - mtilde_f**2/2)/(Htilde*mtilde_f**2)

In [18]:
g = 9.81

In [19]:
ce = symbols("c_e")

In [20]:
ce

c_e

In [21]:
te = (ce/g)*m/T

In [22]:
te

0.101936799184506*c_e*mtilde_f/Ttilde

In [23]:
h = 100000/(te*te*g)

In [24]:
h

981000.0*Ttilde**2/(c_e**2*mtilde_f**2)

In [25]:
sH = sh.subs(H, h)

In [26]:
sH

1.01936799184506e-6*c_e**2*(Ttilde**2*log(1 - mtilde_f)**2/2 + Ttilde*mtilde_f + Ttilde*log(1 - mtilde_f) - mtilde_f**2/2)/Ttilde**2

In [27]:
f = lambdify((T, m, ce), sH, "numpy")

In [28]:
f(2, 0.65, 2500)

1.9007370304161637

In [29]:
def penal(T, mf):
    return (T + 5*mf)

In [30]:
Tr = np.linspace(1.1, 3, 10)
mf = np.linspace(0.1, 0.8, 10)
ce = 2500

Tbest = 3
mfbest = 0.7
sbes = 0

for i in range(len(Tr)):
    r = []
    for k in range(len(mf)):
        if f(Tr[i], mf[k], ce) > 1.0:
            if penal(Tbest, mfbest) > penal(Tr[i], mf[k]):
                Tbest = Tr[i]
                mfbest = mf[k]
                sbes = f(Tr[i], mf[k], ce)
                
print(Tbest, mfbest, sbes)

1.5222222222222224 0.6444444444444445 1.2046225358951406
